In [59]:
from glob import glob
import pandas as pd
import re

In [43]:
results_corpus = glob("../data/results/gpt-j-corpus-results/*.txt")

In [44]:
corpus_validation = pd.read_csv("../data/corpus_valid.csv", nrows=1340)

In [56]:
def merge_paths_dataset(paths, dataset):

    paths_with_ids = [(int(path.split("_")[-1].split(".")[0]), path) for path in paths]
    paths_with_ids.sort(key=lambda x: x[0])
    values= []

    for _, path in paths_with_ids:
        with open(path, "r") as f:
            lines = f.readlines()
            values.append(lines[21])

    dataset["prompt"] = values
    return dataset


corpus_validation_merged = merge_paths_dataset(results_corpus, corpus_validation)

In [67]:
def extract_prompt(x):
    match = re.match(".*<(.*)>", x)

    return match[1] if match else "not-found"



corpus_validation_merged['extracted_label'] = corpus_validation_merged["prompt"].apply(extract_prompt)

In [69]:
corpus_validation_merged['extracted_label'].value_counts()

other            715
common-ground    617
not-found          6
title              1
sound              1
Name: extracted_label, dtype: int64

In [74]:
corpus_validation_filtered = corpus_validation_merged[corpus_validation_merged['extracted_label'].isin(["not-found", "sound"]) == False]

In [75]:
corpus_validation_filtered["extracted_label"].value_counts()

other            715
common-ground    617
title              1
Name: extracted_label, dtype: int64

In [77]:
from sklearn.metrics import classification_report

print(classification_report(corpus_validation_filtered["label"], corpus_validation_filtered["extracted_label"]))

               precision    recall  f1-score   support

     anecdote       0.00      0.00      0.00       221
   assumption       0.00      0.00      0.00       899
common-ground       0.03      0.53      0.06        36
        other       0.01      0.77      0.03        13
   statistics       0.00      0.00      0.00        32
    testimony       0.00      0.00      0.00       109
        title       1.00      0.04      0.08        23

     accuracy                           0.02      1333
    macro avg       0.15      0.19      0.02      1333
 weighted avg       0.02      0.02      0.00      1333



/home/georg/.cache/pypoetry/virtualenvs/ltp-project-JPCASTvD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/georg/.cache/pypoetry/virtualenvs/ltp-project-JPCASTvD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/georg/.cache/pypoetry/virtualenvs/ltp-project-JPCASTvD-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_